## 1. Cargar los datos

Vamos a empezar cargando los datos de artistas, del fichero **artists.dat**. Para facilitar el visualizado de los datos ordenaremos alfabéticamente los grupos.

In [1]:
import pandas as pd
import csv
pd.set_option('display.max_rows', None, 'display.max_columns', None)


In [2]:
df = pd.read_csv('artists.dat', header=0, sep='\t', usecols=[0,1])
df.rename(columns={'id': 'artistID'}, inplace=True)
df = df.sort_values('name')
df = df.reset_index(drop=True)

assert list(df['artistID'][df['name'] == 'MALICE MIZER'])[0] == 1
assert list(df['artistID'][df['name'] == 'Katatonia'])[0] == 370
assert list(df['artistID'][df['name'] == 'Eluveitie'])[0] == 837
assert list(df['artistID'][df['name'] == 'Windy & Carl'])[0] == 6155
assert list(df['artistID'][df['name'] == 'Emilie Autumn'])[0] == 5079
assert list(df['artistID'][df['name'] == 'Hamlet'])[0] == 10302
assert list(df['artistID'][df['name'] == 'Goot'])[0] == 14011

## 2. Visualizar los datos

In [3]:
pd.set_option('display.max_rows', None, 'display.max_columns', None) #Permite mostrar todas las filas del dataframe
df.head()

,artistID,name
0,14846,!!!
1,10635,!DISTAIN
2,8209,!deladap
3,759,#####
4,10004,#2 Orchestra


## 3. Limpieza de datos y Data Enrichment

En este apartado vamos a atajar principalmente dos puntos:
1. Realizar una limpieza de artistas, usando el número de usuarios del que disponemos información y el número de oyentes. Este último lo recopilaremos usando la API que pone last.FM a nuestra disposición.
2. Por otra parte, asignaremos a cada artista tags o etiquetas, que corresponderán principalmente a géneros musicales. Para ello usaremos también la API de last.FM, y realizaremos una selección de tags manual y una posterior limpieza, de manera que se use un número limitado de etiquetas y además con la seguridad de que son válidas y útiles.


### 3.1 Purga de grupos

Queremos:

- Comprobar que no hay grupos repetidos (mismo grupo, distinto formato de string, mayúsculas, minúsculas, símbolos o abreviaturas).  
- Comprobar que no hay grupos incorrectos: fechas, títulos de vídeos de youtube, colaboraciones o símbolos inválidos.
- Eliminar grupos poco relevantes (con un número muy bajo de oyentes).

Para ello, eliminar los artistas con menos de X 'listeners' usando la API de last.fm parece ser una opción viable, ya que los grupos incorrectos suelen tener un número bajo de oyentes.

https://www.last.fm/api/show/artist.getInfo

Get the metadata for an artist. Includes biography, truncated at 300 characters.

**artist (Required (unless mbid)] :** The artist name  
    **mbid (Optional) :** The musicbrainz id for the artist  
    **lang (Optional) :** The language to return the biography in, expressed as an ISO 639 alpha-2 code.  
    **autocorrect[0|1] (Optional) :** Transform misspelled artist names  into correct artist names, returning the correct version instead.   The corrected artist name will be returned in the response.  
    **username (Optional) :** The username for the context of the request. If supplied, the user's playcount for this artist is included in the response.  
**api_key (Required) :** A Last.fm API key.


In [4]:
def artist_getInfo(artist, api_key='3675d2cb82f8d59bcc885774869aca18', mbid=None, lang=None, autocorrect=None, username=None,  maxTags = 10):
    import http.client,  urllib.parse, json

    #Parametros necesarios
    params_dict = {
            'method':'artist.getInfo',
            'artist' : artist,
            'api_key' : api_key,
            'format' : 'json'
    }

    #Parametros opcionales
    if mbid:
        params_dict['mbid'] = mbid
    if lang:
        params_dict['lang'] = lang
    if autocorrect:
        params_dict['autocorrect'] = autocorrect
    if mbid:
        params_dict['username'] = username
    

    params = urllib.parse.urlencode(params_dict) #Parametros de la llamada
    conn = http.client.HTTPConnection('ws.audioscrobbler.com') #URL de la API
    conn.request('GET', '/2.0/', params, {}) #Llamada a la API
    res = conn.getresponse() #Obtenemos la respuesta
    body = json.loads(res.read().decode('utf-8')) #Leemos el body y lo convertimos en dict
    conn.close() #Cerramos la conexion

    #Extras
    #headers = res.getheaders() #Leemos los headers
    #code = res.getcode() #Leemos el codigo (200 Ok)
    #reason = res.reason #Leemos la descripcion del codigo
    if 'artist' in body.keys():
        listeners = int(body['artist']['stats']['listeners'])
    else:
        listeners = -1
        
    return listeners

Llamamos a la API tantas veces como sea necesario, ya que en ocasiones falla la conexión y se interrumpe el proceso. Para ello, lo único que debemos hacer es copiar el contenido conseguido hasta ahora ejecutando la celda inferior, y a continuación cambiar el índice 'idx' para seguir por donde lo hemos dejado.

Realmente podríamos hacerlo todo seguido y copiarlo al final, pero debido a lo lento del proceso, me parece más conveniente y seguro ir copiando en el fichero de texto a medida que conseguimos los datos. 

Por seguridad, prefiero añadirle un entero aleatorio al inicio del fichero, por si ejecutamos accidentalmente la celda.

In [47]:
#import random
#number = random.randint(0,1000000)
#with open(str(number)+"artist_listeners.dat", "a") as f:
#    writer = csv.writer(f, delimiter='\t')
#    writer.writerow(('artistID', 'name', 'listeners'))

En total deberemos ejecutar las dos celdas de a continuación unas 10 veces. 

In [ ]:
#artist_listeners = []
#idx = 0
#for index, row in df[idx:].iterrows():
#    print(index)
#    artist_listeners.append((row['id'], row['name'], artist_getInfo(row['name'])))

In [ ]:
#with open(str(number)+"artist_listeners.dat", "a") as f:
#    writer = csv.writer(f, delimiter='\t')
#    writer.writerows(artist_listeners)

Ahora que ya hemos conseguido el número de oyentes de cada grupo, vamos a ordenarlos de más a menos oyentes, y vamos a **decidir** cual es **el umbral** que nos resulta más adecuado para descartar los artistas.

Deberemos repetir los pasos vistos en el cargado de datos para el nuevo fichero construido.

In [5]:
import pandas as pd

#Cargamos los datos
df_listeners = pd.read_csv('artist_listeners.dat', header=0, sep='\t')
#Los ordenamos
df_listeners = df_listeners.sort_values('listeners', ascending=False).reset_index(drop=True)
#Los visualizamos
pd.set_option('display.max_rows', None, 'display.max_columns', None)
display(df_listeners.head())

,artistID,name,listeners
0,65,Coldplay,5369132
1,154,Radiohead,4718748
2,220,Red Hot Chili Peppers,4608144
3,288,Rihanna,4542147
4,475,Eminem,4504111


También vamos a usar el número de usuarios del que disponemos datos para cada grupo, por lo que vamos a cargar el fichero **user_artists.dat** y vamos a contar las veces que aparece cada artista.

In [6]:
import pandas as pd

#Cargamos los datos
df_ua = pd.read_csv('user_artists.dat', header=0, sep='\t', usecols=[0,1])
#Los visualizamos
pd.set_option('display.max_rows', None, 'display.max_columns', None)
display(df_ua.head())

,userID,artistID
0,2,51
1,2,52
2,2,53
3,2,54
4,2,55


In [102]:
df_freq = df_ua[['artistID']].copy()
df_freq = df_freq.groupby(['artistID']).size().reset_index()
df_freq.rename(columns = {0: 'freq'}, inplace = True)
df_freq = df_freq.sort_values('freq', ascending=False)
df_freq.head()

,artistID,freq
83,89,611
283,289,522
282,288,484
221,227,480
294,300,473


Ahora que ya tenemos tanto el número de oyentes como el número de veces que aparece el artista en nuestros datos, vamos a construir un nuevo dataframe que reuna ambos campos.

In [8]:
df_freq_listeners = df_freq.join(df_listeners.set_index('artistID'), on='artistID').reset_index(drop=True)
df_freq_listeners = df_freq_listeners.sort_values('listeners', ascending=False).reset_index(drop=True)
df_freq_listeners = df_freq_listeners[['artistID', 'name', 'freq', 'listeners']]
df_freq_listeners.head()

,artistID,name,freq,listeners
0,65,Coldplay,369,5369132
1,154,Radiohead,393,4718748
2,220,Red Hot Chili Peppers,222,4608144
3,288,Rihanna,484,4542147
4,475,Eminem,204,4504111


Y ahora sí vamos a decidir qué umbrales aplicamos. Podemos jugar con los dos campos, frequency y listeners, y ver qué nos resulta más conveniente.
Después de hacer varias combinaciones y de analizar un poco los datos, creo que lo más sensato es eliminar los artistas de los que no disponemos datos de al menos de 3 usuarios y de los muy poco relevantes (menos de 5000 oyentes).

Por qué **eliminar los que tienen menos de 3 usuarios en nuestros datos**? Porque cuando tengamos que hacer los ejes entre grupos usaremos el número de usuarios en común. En el caso de los artistas con 1 solo usuario, tendrán 100% de usuarios en común con todo el resto de artistas de ese usuario. En el caso de los artistas con 2 usuarios, tendremos exactamente el mismo problema si ponemos el umbral de unión a menos del 50%. Por lo tanto, creo que tener en cuenta esos artistas puede ensuciar los datos y causar resultados erróneos.

Por qué **eliminar los artistas muy poco relevantes**? Porque esto además nos asegura que no hay artistas con nombres incorrectos, suelen tener entre 0 y 100 oyentes (o -1 en el caso de los artistas no encontrados). No queremos ruido en nuestros datos.


In [9]:
df_purga = df_freq_listeners[~((df_freq_listeners.freq < 3) | (df_freq_listeners.listeners < 5000))]
df_purga = df_purga.sort_values('freq', ascending=False).reset_index(drop=True)
df_purga.head()

,artistID,name,freq,listeners
0,89,Lady Gaga,611,3798959
1,289,Britney Spears,522,3243214
2,288,Rihanna,484,4542147
3,227,The Beatles,480,3658694
4,300,Katy Perry,473,3721531


In [11]:
len(df_purga.index)

4562

In [12]:
assert list(df_purga['artistID'][df_purga['name'] == 'MALICE MIZER'])[0] == 1
assert list(df_purga['artistID'][df_purga['name'] == 'Katatonia'])[0] == 370
assert list(df_purga['artistID'][df_purga['name'] == 'Eluveitie'])[0] == 837
assert list(df_purga['artistID'][df_purga['name'] == 'Windy & Carl'])[0] == 6155
assert list(df_purga['artistID'][df_purga['name'] == 'Emilie Autumn'])[0] == 5079

In [13]:
import csv
df_purga.to_csv("artists-v2.dat", sep='\t', index=False)

### 3.2 Obtención de tags

Para empezar vamos a obtener los 10 tags más frecuentes de cada artista. Para ello hemos creado una función que nos permite recopilar los tags más frecuentes de cada artista (10 por defecto). Esta función puede modificarse para que devuelva otra información si se desea.

https://www.last.fm/api/show/artist.getTopTags

Get the top tags for an artist on Last.fm, ordered by popularity.  

**artist (Required (unless mbid)] :** The artist name  
**mbid (Optional) :** The musicbrainz id for the artist  
**autocorrect[0|1] (Optional) :** Transform misspelled artist names into correct artist names, returning the correct   version instead. The corrected artist name will be returned in the response.  
**api_key (Required) :** A Last.fm API key.  

In [123]:
def artist_getTopTags(artist, api_key='3675d2cb82f8d59bcc885774869aca18', mbid=None, autocorrect=None, maxTags = 10):
    import http.client,  urllib.parse, json

    #Parametros necesarios
    params_dict = {
            'method':'artist.getTopTags',
            'artist' : artist,
            'api_key' : api_key,
            'format' : 'json'
    }

    #Parametros opcionales
    if autocorrect:
        params_dict['autocorrect'] = autocorrect
    if mbid:
        params_dict['mbid'] = mbid

    params = urllib.parse.urlencode(params_dict) #Parametros de la llamada
    conn = http.client.HTTPConnection('ws.audioscrobbler.com') #URL de la API
    conn.request('GET', '/2.0/', params, {}) #Llamada a la API
    res = conn.getresponse() #Obtenemos la respuesta
    body = json.loads(res.read().decode('utf-8')) #Leemos el body y lo convertimos en dict
    conn.close() #Cerramos la conexion

    #Extras
    #headers = res.getheaders() #Leemos los headers
    #code = res.getcode() #Leemos el codigo (200 Ok)
    #reason = res.reason #Leemos la descripcion del codigo
    tags = []
    #Obtenemos únicamente los tags
    i = 0
    for d in body['toptags']['tag']:
        if i == maxTags:
            break;
        tag = d['name']
        tags.append(tag.lower())
        i+=1
        
    return tags

RHCP_tags = artist_getTopTags('Red Hot Chili Peppers')

In [124]:
str(RHCP_tags)

"['rock', 'alternative rock', 'alternative', 'funk rock', 'funk', 'seen live', 'red hot chili peppers', 'punk', 'american', 'indie']"

Antes de nada, cargamos en un dataframe el nuevo archivo *artists.dat* obtenido en el apartado anterior.

In [67]:
df = pd.read_csv('artists-v2.dat', header=0, sep='\t', usecols=[0,1])

Y repetimos los mismos pasos que para la obtención del número de oyentes, pero esta vez para las etiquetas de los artistas,

In [95]:
#import csv
#import random
#number = random.randint(0,1000000)
#with open(str(number)+"artist_tags.dat", "a") as f:
#    writer = csv.writer(f, delimiter='\t')
#    writer.writerow(('artistID', 'name', 'tag1', 'tag2', 'tag3', 'tag4', 'tag5', 'tag6', 'tag7', 'tag8', 'tag9', 'tag10'))
#    
#
#with open(str(number)+"tags.dat", "a") as f:
#    writer = csv.writer(f)
#    writer.writerow(('tag'))   

In [101]:
#artist_tags = []
#all_tags = []
#idx = 3531
#for index, row in df[idx:].iterrows():
#    print(index)
#    tags_list = artist_getTopTags(row['name']) 
#    artist_tags.append([row['artistID'],row['name']]+tags_list)
#    all_tags = all_tags+tags_list

Debido a que posteriormente necesitaré contabilizar el número de apariciones de cada tag para hacer una limpieza de tags, he optado por hacer otro archivo complementario, a medida que realizo el de artistas y etiquetas, que contiene únicamente las etiquetas.

In [100]:
#with open(str(number)+"artist_tags.dat", "a") as f:
#    writer = csv.writer(f, delimiter='\t')
#    writer.writerows(artist_tags)
#    
#with open(str(number)+"tags.dat", "a") as f:
#    writer = csv.writer(f, delimiter='\n')
#    writer.writerow(all_tags)

Una vez ya tenemos los dos archivos, **artist_tags.dat** y **tags.dat**, los cargamos para trabajar con ellos.

In [3]:
df_artist_tags = pd.read_csv('artist_tags.dat', header=0, sep='\t')
df_tags = pd.read_csv('tags.dat', header=0)

Calculamos la frecuencia de cada tag y los ordenamos de más a menos.

In [24]:
df_tags_freq = df_tags.groupby(['tag']).size().reset_index()
df_tags_freq.rename(columns = {0: 'freq'}, inplace = True)
df_tags_freq = df_tags_freq.sort_values('freq', ascending=False).reset_index(drop=True)

Creamos un nuevo fichero **tags_freq.dat** con los tags y sus frecuencias.

In [91]:
df_tags_freq.head()
df_tags_freq.to_csv("tags_freq.dat", sep='\t', index=False)

Manualmente he realizado una selección de tags de los 420 tags más frecuentes. Por un lado están los tags a eliminar **tags_to_delete.dat**, que son tags poco útiles o incorrectos. La mayoría corresponden a nacionalidades u otras características del artista no relevantes.  
Por otro lado estan los tags a sustituir **tags_to_replace.dat**, que son tags incorrectos pero que pueden substituirse por un tag correcto, como géneros repetidos escritos de distinta forma (hip hop en lugar de hip-hop) o géneros que se han acompañado con una nacionalidad, sin tener distinción alguna del género original (*rock brasileño* o *rock ruso* en lugar de simplemente *rock*).

In [4]:
df_tags = pd.read_csv('tags_freq.dat', header=0, sep='\t', usecols =[0])
df_tags_replace = pd.read_csv('tags_to_replace.dat', header=0, sep='\t')
#df_tags_delete = pd.read_csv('tags_to_delete.dat', header=0)
df_tags_delete = pd.read_csv('tags_to_delete-v2.dat', header=0)
df_tags = df_tags[:420]

In [5]:
df_tags.head()

,tag
0,seen live
1,rock
2,alternative
3,pop
4,indie


In [6]:
df_tags_replace.head()

,incorrect tag,tag
0,abstract hip-hop,hip-hop
1,alternative rap,alternative hip-hop
2,avantgarde,avant-garde
3,brazilian rock,rock
4,chanson francaise,chanson


In [7]:
df_tags_delete.head()

,invalid tag
0,00s
1,4ad
2,50s
3,60s
4,70s


Ahora en lugar de los tags que queremos eliminar, queremos los tags válidos. Para ello eliminamos los tags incorrectos de la lista de tags.

In [8]:
for index, tag in df_tags_delete.iterrows():
    df_tags = df_tags.loc[~(df_tags == tag[0]).sum(axis=1).astype(bool)]
#df_tags.to_csv("tags-v2.dat", index=False)
df_tags.to_csv("tags-v3.dat", index=False)

Ya tenemos los tags válidos **df_tags** y los tags a reemplazar **df_tags_replace**. Ahora por fin vamos a realizar la limpieza, creando un nuevo dataframe que contenga únicamente los tags válidos seleccionados.

In [9]:
df_tags = pd.read_csv('tags-v3.dat', header=0)
df_tags = df_tags.sort_values('tag').reset_index(drop=True)

In [10]:
df_tags

,tag
0,8-bit
1,acid jazz
2,acoustic rock
3,aggrotech
4,alt-country
5,alternative
6,alternative hip-hop
7,alternative metal
8,alternative rock
9,ambient


In [11]:
df_artist_tags.head()

,artistID,name,tag1,tag2,tag3,tag4,tag5,tag6,tag7,tag8,tag9,tag10
0,89,lady gaga,pop,dance,electronic,female vocalists,female vocalist,lady gaga,electropop,seen live,american,gaga
1,289,britney spears,pop,dance,female vocalists,britney spears,legend,american,seen live,sexy,female,90s
2,288,rihanna,pop,rnb,female vocalists,dance,hip-hop,rihanna,r&b,seen live,reggae,hip hop
3,227,the beatles,classic rock,rock,british,60s,pop,psychedelic,the beatles,oldies,psychedelic rock,beatles
4,300,katy perry,pop,female vocalists,pop rock,indie,rock,katy perry,american,dance,seen live,female vocalist


Vamos a crear una lista de listas, donde cada lista será una fila del nuevo dataframe. Cada fila será una fila del dataframe df_artist_tags manteniendo únicamente los tags válidos y/o reemplazados.

In [12]:
valid_tags = df_tags.values
replace_tags = df_tags_replace['incorrect tag'].values
dictionary = df_tags_replace.set_index('incorrect tag').T.to_dict('records')[0]
rows = []
for index, data in df_artist_tags.iterrows():
    artistID = data[0]
    name = data[1]
    tags = list(data[2:])
    row = list(data[:2])
    row_tags = []
    valid_tag = None
    for tag in tags:
        if tag in replace_tags:
            valid_tag = dictionary[tag]
        elif tag in valid_tags:
            valid_tag = tag
        if (valid_tag is not None) and (valid_tag not in row_tags):
            row_tags.append(valid_tag)
    row = row+row_tags
    rows.append(row)

/home/luna/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  This is separate from the ipykernel package so we can avoid doing imports until


Por último, escribimos en el fichero **artist_tags-v2.dat** las nuevas filas del dataframe. 
Hemos conseguido obtener un fichero con únicamente los artistas y tags válidos seleccionados.

In [13]:
with open("artist_tags-v2.dat", "w") as f:
    writer = csv.writer(f, delimiter='\t')
    writer.writerow(('artistID', 'name', 'tag1', 'tag2', 'tag3', 'tag4', 'tag5', 'tag6', 'tag7', 'tag8', 'tag9', 'tag10'))    

with open("artist_tags-v2.dat", "a") as f:
    writer = csv.writer(f, delimiter='\t')
    writer.writerows(rows)

In [14]:
df_artist_tags = pd.read_csv('artist_tags-v2.dat', sep = '\t', header=0)

In [15]:
df_artist_tags.head()

,artistID,name,tag1,tag2,tag3,tag4,tag5,tag6,tag7,tag8,tag9,tag10
0,89,lady gaga,pop,dance,electronic,electropop,NaN,NaN,NaN,NaN,NaN,NaN
1,289,britney spears,pop,dance,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,288,rihanna,pop,rnb,dance,hip-hop,reggae,NaN,NaN,NaN,NaN,NaN
3,227,the beatles,classic rock,rock,pop,psychedelic,psychedelic rock,NaN,NaN,NaN,NaN,NaN
4,300,katy perry,pop,pop rock,indie,rock,dance,NaN,NaN,NaN,NaN,NaN


In [16]:
assert list(df_artist_tags['artistID'][df_artist_tags['name'] == 'malice mizer'])[0] == 1
assert list(df_artist_tags['artistID'][df_artist_tags['name'] == 'katatonia'])[0] == 370
assert list(df_artist_tags['artistID'][df_artist_tags['name'] == 'eluveitie'])[0] == 837
assert list(df_artist_tags['artistID'][df_artist_tags['name'] == 'windy & carl'])[0] == 6155
assert list(df_artist_tags['artistID'][df_artist_tags['name'] == 'emilie autumn'])[0] == 5079